# Batch Processing LLM Prompts / Pakešapstrāde LLM uzvednēm

In [ ]:
#@title 📦 Batch LLM Processor (Hidden Code by Default, with Sampling)
#@markdown **Instructions:**  
#@markdown 1. Enter your **OpenRouter API Key**.  
#@markdown 2. Enter a **System Prompt** (optional).  
#@markdown 3. Choose or type a **Model ID**.  
#@markdown 4. Choose how to provide your ZIP file:  
#@markdown    - **Upload (simple, <100 MB)**  
#@markdown    - **Google Drive path (good for larger files)**  
#@markdown 5. (Optional) Tick the box to include inputs in results.  
#@markdown 6. (Optional) Limit number of files processed and set a random seed.  
#@markdown 7. Click **Run Processing** and wait for the download link.

import os, zipfile, base64, mimetypes, json, time, random
import pandas as pd
import requests
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files as colab_files, drive

# --- UI Controls ---
api_key = widgets.Password(description="API Key:", layout=widgets.Layout(width="400px"))
system_prompt = widgets.Textarea(description="System Prompt:", layout=widgets.Layout(width="500px", height="80px"))
model_dropdown = widgets.Dropdown(
    options=[
        ("Gemini 2.5 Flash (default)", "google/gemini-2.5-flash"),
        ("Gemini 2.5 Pro", "google/gemini-2.5-pro"),
        ("GPT-5", "openai/gpt-5"),
        ("Claude Sonnet", "anthropic/claude-sonnet-4"),
        ("Mistral Medium 3.1", "mistralai/mistral-medium-3.1"),
    ],
    value="google/gemini-2.5-flash",
    description="Model:"
)
model_custom = widgets.Text(description="Custom Model:", placeholder="e.g. openai/gpt-4.1-mini", layout=widgets.Layout(width="400px"))
include_inputs = widgets.Checkbox(value=False, description="Include inputs in output zip")

upload_method = widgets.RadioButtons(
    options=["Upload (<100MB)", "Google Drive path"],
    description="Upload via:"
)

upload_btn = widgets.FileUpload(accept=".zip", multiple=False, description="Upload ZIP")
drive_path = widgets.Text(description="Drive path:", placeholder="/content/drive/MyDrive/myfile.zip", layout=widgets.Layout(width="500px"))

# New sampling options
max_files = widgets.IntText(value=0, description="Max files (0=all):")
seed_value = widgets.IntText(value=2025, description="Seed:")

run_btn = widgets.Button(description="Run Processing", button_style="success")

ui_box = widgets.VBox([api_key, system_prompt, model_dropdown, model_custom,
                       include_inputs, upload_method, upload_btn, drive_path,
                       max_files, seed_value, run_btn])
display(ui_box)

# --- Backend function ---
def process_zip(api_key, system_prompt, model, zip_path, include_inputs, max_files, seed_value):
    job_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    work_dir = f"job_{job_id}"
    os.makedirs(work_dir, exist_ok=True)

    input_dir = os.path.join(work_dir, "input")
    os.makedirs(input_dir, exist_ok=True)

    with zipfile.ZipFile(zip_path, "r") as zf:
        all_files = [f for f in zf.namelist() if not f.endswith("/")]

        # Random sampling if requested
        if max_files > 0 and max_files < len(all_files):
            random.seed(seed_value)
            files_list = random.sample(all_files, max_files)
        else:
            files_list = all_files

        # Extract only the needed files
        for fname in files_list:
            zf.extract(fname, input_dir)

    rows = []
    total = len(files_list)

    for idx, fname in enumerate(files_list, start=1):
        fpath = os.path.join(input_dir, fname)
        ext = os.path.splitext(fname)[1].lower()
        user_content = []

        if ext in [".txt", ".md"]:
            with open(fpath, "r", encoding="utf-8") as f:
                text = f.read()
            user_content.append({"type": "text", "text": text})

        elif ext in [".jpg", ".jpeg", ".png", ".tif", ".tiff"]:
            mime, _ = mimetypes.guess_type(fpath)
            if mime is None:
                mime = "image/png"
            with open(fpath, "rb") as img_file:
                img_b64 = base64.b64encode(img_file.read()).decode("utf-8")
            user_content.append({"type": "text", "text": f"Please analyze image: {fname}"})
            user_content.append({"type": "image_url", "image_url": {"url": f"data:{mime};base64,{img_b64}"}})
        else:
            rows.append({"file": fname, "output": "Unsupported file type"})
            continue

        payload = {
            "model": model,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ]
        }
        headers = {"Authorization": f"Bearer {api_key}"}

        try:
            r = requests.post("https://openrouter.ai/api/v1/chat/completions",
                              json=payload, headers=headers, timeout=120)
            r.raise_for_status()
            reply = r.json()["choices"][0]["message"]["content"]
        except Exception as e:
            reply = f"ERROR: {e}"

        rows.append({"file": fname, "output": reply})
        clear_output(wait=True)
        print(f"Processed {idx}/{total}: {fname}")

        time.sleep(0.2)

    # Save outputs
    output_csv = os.path.join(work_dir, "output.csv")
    pd.DataFrame(rows).to_csv(output_csv, index=False)

    # Save input.csv with metadata
    input_rows = []
    for fname in files_list:
        fpath = os.path.join(input_dir, fname)
        fsize = os.path.getsize(fpath)
        ftype = mimetypes.guess_type(fpath)[0] or "unknown"
        input_rows.append({"file_name": fname, "full_path": fpath, "file_type": ftype, "file_size": fsize})
    pd.DataFrame(input_rows).to_csv(os.path.join(work_dir, "input.csv"), index=False)

    meta = {
        "model": model,
        "system_prompt": system_prompt,
        "submitted_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "include_inputs": include_inputs,
        "max_files": max_files,
        "seed": seed_value
    }
    with open(os.path.join(work_dir, "meta.json"), "w") as f:
        json.dump(meta, f, indent=2)

    # Create results zip
    zip_out = f"results_{job_id}.zip"
    with zipfile.ZipFile(zip_out, "w", zipfile.ZIP_DEFLATED) as zf:
        zf.write(output_csv, arcname="output.csv")
        zf.write(os.path.join(work_dir, "input.csv"), arcname="input.csv")
        zf.write(os.path.join(work_dir, "meta.json"), arcname="meta.json")
        if include_inputs:
            for fname in os.listdir(input_dir):
                fpath = os.path.join(input_dir, fname)
                zf.write(fpath, arcname=os.path.join("input", fname))

    return zip_out

# --- Hook up Run button ---
def on_run_clicked(b):
    if upload_method.value == "Upload (<100MB)":
        if not upload_btn.value:
            print("⚠️ Please upload a ZIP file first.")
            return
        fname = list(upload_btn.value.keys())[0]
        zip_path = "input_uploaded.zip"
        with open(zip_path, "wb") as f:
            f.write(upload_btn.value[fname]["content"])
    else:
        if not drive_path.value.strip():
            print("⚠️ Please provide a valid Google Drive path.")
            return
        zip_path = drive_path.value.strip()
        if not os.path.exists(zip_path):
            drive.mount('/content/drive')
            if not os.path.exists(zip_path):
                print(f"⚠️ File not found: {zip_path}")
                return

    chosen_model = model_custom.value.strip() if model_custom.value else model_dropdown.value
    result_zip = process_zip(api_key.value, system_prompt.value, chosen_model, zip_path,
                             include_inputs.value, max_files.value, seed_value.value)
    print("✅ Processing complete. Download your results:")
    colab_files.download(result_zip)

run_btn.on_click(on_run_clicked)


Python version: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
Python executable: c:\Users\vsaules\Github\PublicTools\venv\Scripts\python.exe
Current date and time: 2025-10-01 15:24:00.648220
